https://medium.com/@whyamit404/what-is-pandas-json-normalize-and-why-use-it-50ae0cf2d12d

In [1]:
import json
from IPython.display import display, JSON

# Example JSON data
data = {
    "name": "John Doe",
    "age": 30,
    "isStudent": False,
    "courses": [
        {"title": "Mathematics", "credits": 3},
        {"title": "Physics", "credits": 4}
    ],
    "address": {
        "street": "123 Main St",
        "city": "Anytown",
        "zip": "12345"
    }
}

# Display the JSON data as a collapsible tree
display(JSON(data, expanded=True)) # Set expanded=False for initially collapsed view

<IPython.core.display.JSON object>

In [2]:
import pandas as pd
from pprint import pprint
import json

data = [
    {"id": 1, "name": "John", "info": {"age": 30, "city": "New York"}},
    {"id": 2, "name": "Jane", "info": {"age": 25, "city": "London"}}
]
# pprint(json.dumps(data))
display(JSON(data, expanded=True)) # Set expanded=False for initially collapsed view

<IPython.core.display.JSON object>

In [3]:
df = pd.json_normalize(data)
print(df)

   id  name  info.age info.city
0   1  John        30  New York
1   2  Jane        25    London


In [4]:
data = {
    "school": "ABC High",
    "students": [
        {"name": "Tom", "scores": {"math": 90, "science": 85}},
        {"name": "Sara", "scores": {"math": 95, "science": 92}}
    ]
}

In [5]:
df = pd.json_normalize(data)
df

,school,students
0,ABC High,"[{'name': 'Tom', 'scores': {'math': 90, 'scien..."


In [6]:
df = pd.json_normalize(data, meta=['school'])
df

,school,students
0,ABC High,"[{'name': 'Tom', 'scores': {'math': 90, 'scien..."


In [7]:
df = pd.json_normalize(data, record_path=['students'])
df

,name,scores.math,scores.science
0,Tom,90,85
1,Sara,95,92


The students list is buried inside the JSON structure.

record_path=['students'] digs into that list and flattens it.

meta=['school'] pulls in the school name for every student.

You might be wondering: “What if I didn’t use meta?”
Well, you’d lose the school info entirely because it’s outside the students list. That’s why meta is crucial when you want to retain context.



In [8]:
df = pd.json_normalize(data, record_path=['students'], meta=['school'])
df

,name,scores.math,scores.science,school
0,Tom,90,85,ABC High
1,Sara,95,92,ABC High


In [9]:
df = pd.json_normalize(data['students'], sep='_')
print(df)

   name  scores_math  scores_science
0   Tom           90              85
1  Sara           95              92


How to Flatten Multiple Levels of Nested JSON?

You might be wondering, “What if my JSON is a real labyrinth with layers within layers?”
No worries — I’ve got you covered.

When dealing with deeply nested JSON, the key is to use:

record_path: This helps you dig into lists nested deep inside.

meta: This grabs any extra information outside of the list.

max_level: If you only want to flatten up to a certain depth, this parameter keeps things under control.


In [10]:
import pandas as pd

data = {
    "company": "TechCorp",
    "departments": [
        {
            "name": "Engineering",
            "employees": [
                {"name": "Alice", "skills": {"Python": 90, "SQL": 80}},
                {"name": "Bob", "skills": {"Python": 85, "SQL": 88}}
            ]
        },
        {
            "name": "HR",
            "employees": [
                {"name": "Charlie", "skills": {"Recruitment": 95, "Onboarding": 89}}
            ]
        }
    ]
}

In [15]:
# Flattening nested lists with meta info
df = pd.json_normalize(
    data,
    record_path=['departments', 'employees'],
    meta=[['departments', 'name'], 'company'],
    sep='_'
)

print(df)

      name  skills_Python  skills_SQL  skills_Recruitment  skills_Onboarding  \
0    Alice           90.0        80.0                 NaN                NaN   
1      Bob           85.0        88.0                 NaN                NaN   
2  Charlie            NaN         NaN                95.0               89.0   

  departments_name   company  
0      Engineering  TechCorp  
1      Engineering  TechCorp  
2               HR  TechCorp  


NaN values are often not errors—they’re just missing data.

Here’s why it happens:

Some keys might be present in one record but missing in another.
This inconsistency leads to NaN (which stands for “Not a Number”).

How to Handle It?

Ignore Missing Keys:

Use errors='ignore' to skip over missing fields gracefully.

In [24]:
data = [
    {"id": 1, "name": "John", "info": {"age": 30}},
    {"id": 2, "name": "Jane"}  # Missing 'info'
]

In [25]:
df = pd.json_normalize(data)
print(df)

   id  name  info.age
0   1  John      30.0
1   2  Jane       NaN


In [26]:
df = pd.json_normalize(data, errors='ignore')
print(df)

   id  name  info.age
0   1  John      30.0
1   2  Jane       NaN


In [27]:
df.fillna("Not Available", inplace=True)
print(df)

   id  name       info.age
0   1  John           30.0
1   2  Jane  Not Available


/var/folders/mm/h2z28vhs6cdbnsksl2fzl19w0000gn/T/ipykernel_19282/667628441.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not Available' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Not Available", inplace=True)


In [28]:
df = pd.json_normalize(data, errors='ignore')
print(df)

   id  name  info.age
0   1  John      30.0
1   2  Jane       NaN


In [29]:
df = df.fillna("Not Available")
print(df)

   id  name       info.age
0   1  John           30.0
1   2  Jane  Not Available


In [11]:
import requests
import pandas as pd

# Mock API response
response = {
    "users": [
        {"id": 1, "name": "Alice", "contacts": {"email": "alice@example.com"}},
        {"id": 2, "name": "Bob", "contacts": {"email": "bob@example.com"}}
    ]
}

# Convert API response to JSON and normalize
df = pd.json_normalize(response['users'])
print(df)

   id   name     contacts.email
0   1  Alice  alice@example.com
1   2    Bob    bob@example.com


/opt/anaconda3/envs/py311b/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [31]:
df = pd.json_normalize(response['users'], sep='_')
print(df)


   id   name     contacts_email
0   1  Alice  alice@example.com
1   2    Bob    bob@example.com
